# Regression models

In this notebook we show how the tuning library can be called for any regression predictor implementing our `fit` and `evaluate` methods.
The tuning process can be differentiated based on a number of different settings, most importantly the set of parameters to be explored and the method of evaluation. The latter is by default set to train-test split for timing reasons (since GridSearch is a computationally expensive procedure). However more robust results are likely using `method='CV'`

In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.append('..')

from regressors import RidgePredictor, LassoPredictor, RandomForestPredictor
from tuning import tune, bayesian_optimization
from preprocessing import keep_intersection_of_columns

C:\Users\s100385\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [8]:
# load datasets and define writhing path
aggregated_train = pd.read_csv('../data/aggregated_train.csv', dtype={"fullVisitorId": str})
aggregated_test = pd.read_csv('../data/aggregated_test.csv', dtype={"fullVisitorId": str})
write_to = '../data/tuning.txt'

In [3]:
train_x, test_x = keep_intersection_of_columns(aggregated_train.reset_index(drop=True), aggregated_test.reset_index(drop=True))
train_y = np.log(aggregated_train.reset_index(drop=True)["target_sum"]+1)
train_x = train_x.fillna(0).drop(["target_sum", "fullVisitorId"], axis=1)
test_ids = test_x["fullVisitorId"]
test_x = test_x.fillna(0).drop(["target_sum", "fullVisitorId"], axis=1)

In [5]:
param_grid = {
    'max_depth': [3,4,5,6]
}

best_params, best_score = tune(RandomForestPredictor, train_x, train_y, param_grid, silent=False, persist=True, write_to=write_to, method = 'CV')
print("Optimal parameters achieve RMSE = {}".format(best_score))
print("Parameters: {}".format(best_params))

Applying GridSearch for 4 permutations of parameters
Evaluating {'max_depth': 3}
evaluate function took 1162.6 seconds to complete

Evaluating {'max_depth': 4}
evaluate function took 1665.3 seconds to complete

Evaluating {'max_depth': 5}
evaluate function took 1694.5 seconds to complete

Evaluating {'max_depth': 6}
evaluate function took 4540.3 seconds to complete

tune function took 9062.7 seconds to complete

Optimal parameters achieve RMSE = 0.2948202279293129
for parameters:
{'max_depth': 3}


In [ ]:
param_grid = {
    'alpha': np.logspace(-10, -2, 15),
    'normalize': [True], # only false if the data is not normalized!!
}

best_params, best_score = tune(RidgePredictor, train_x, train_y, param_grid, silent=False, persist=True, write_to=write_to, method = 'CV')
print("Optimal parameters achieve RMSE = {}".format(best_score))

In [ ]:
param_grid = {
    'alpha': np.logspace(-10, -2, 15),
    'normalize': [True] # only false if the data is not normalized!!
}

best_params, best_score = tune(LassoPredictor, train_x, train_y, param_grid, silent=False, persist=False, write_to=write_to, method = 'CV')
print("Optimal parameters achieve RMSE = {}".format(best_score))